## Import & Install Statements

In [ ]:
# Install the datasets library for converting the data frame into a Hugging Face data frame.
# !pip install datasets

In [ ]:
# Import statements

from datasets import Dataset
import pandas as pd
import random
import random
import torch

from tqdm import tqdm

from transformers import T5Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, T5ForConditionalGeneration, TrainingArguments, Trainer

In [ ]:
# Read the dataset to pick things back up from where we left them at the end of the third codebase.
finetuning_data = pd.read_csv('/content/finetuning_data_diversified_output_text.csv')

In [ ]:
# Let's take another quick look at the first five rows of the data frame.
finetuning_data

,input_for_model,input_length,input_sentiment,input_sentiment_1_10,input_sentiment_1_100,cleaned_input,adjusted_input_length,cleaned_input_sentiment,cleaned_input_sentiment_1_10,cleaned_input_sentiment_1_100,...,keyword_match_ratio,keyword_match_ratio_1_10,keyword_match_ratio_1_100,politeness_count_input,empathy_count_input,politeness_count_output,empathy_count_output,entity_match_ratio,input_text,output_text
0,"Conversation: Hi Tom, I'm trying to log in to ...",359,0.9955,9.97975,99.77725,conversation hi tom im trying to log in to my ...,223,0.9909,9.95905,99.54955,...,0.063158,5.784211,53.626316,13,4,0,1,0.0,"Customer Issue:\nHi Tom, I'm trying to log in ...","The customer issue is moderately complex, requ..."
1,Conversation: Hi Alex. I recently received an ...,340,0.9924,9.96580,99.62380,conversation hi alex i recently received email...,221,0.9915,9.96175,99.57925,...,0.090909,5.909091,55.000000,9,5,0,0,0.0,Customer Issue:\nHi Alex. I recently received ...,"This is a straightforward issue, best resolved..."
2,"Conversation: Hi Sarah, I am calling because I...",562,0.9969,9.98605,99.84655,conversation hi sarah i calling i unable to cl...,357,0.9935,9.97075,99.67825,...,0.055556,5.750000,53.250000,13,5,0,1,0.0,"Customer Issue:\nHi Sarah, I am calling becaus...","The customer issue is moderately complex, requ..."
3,"Conversation: Hi, I am facing an issue while l...",502,0.9970,9.98650,99.85150,conversation hi i facing issue logging my acco...,334,0.9940,9.97300,99.70300,...,0.057377,5.758197,53.340164,14,7,0,0,0.0,"Customer Issue:\nHi, I am facing an issue whil...","This is a straightforward issue, best resolved..."
4,"Conversation: Hi Sarah, I have an issue with m...",307,0.6212,8.29540,81.24940,conversation hi sarah i have issue with my ord...,206,0.0018,5.50810,50.58910,...,0.091837,5.913265,55.045918,7,3,0,3,0.0,"Customer Issue:\nHi Sarah, I have an issue wit...","The customer issue is moderately complex, requ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,"Conversation: Hi Sarah, I received a mobile ph...",552,0.9952,9.97840,99.76240,conversation hi sarah i received a mobile phon...,398,0.9921,9.96445,99.60895,...,0.112676,6.007042,56.077465,15,12,1,3,0.0,"Customer Issue:\nHi Sarah, I received a mobile...","The customer issue is moderately complex, requ..."
993,"Conversation: Hi, I received an email from Bro...",361,0.9890,9.95050,99.45550,conversation hi i received email brownbox stat...,238,0.9913,9.96085,99.56935,...,0.117647,6.029412,56.323529,10,4,0,1,0.0,"Customer Issue:\nHi, I received an email from ...","The customer issue is moderately complex, requ..."
994,"Conversation: Hi Sarah, I recently purchased a...",493,0.9809,9.91405,99.05455,conversation hi sarah i recently purchased a w...,339,0.9881,9.94645,99.41095,...,0.089552,5.902985,54.932836,12,7,0,2,0.0,"Customer Issue:\nHi Sarah, I recently purchase...","This is a straightforward issue, best resolved..."
995,"Conversation: Hi, I would like to check the st...",448,0.9970,9.98650,99.85150,conversation hi i would like to check the stat...,298,0.9965,9.98425,99.82675,...,0.060150,5.770677,53.477444,18,6,0,1,0.0,"Customer Issue:\nHi, I would like to check the...","The customer issue is moderately complex, requ..."


## Data Preparation for Model Training

In [ ]:
# Let's drop some columns that we don't think will be too useful for model training; while these columns were very useful for EDA & Data Visualization, we don't think they'll be too useful for training the model.
columns_to_drop = ['input_for_model', 'input_length', 'adjusted_input_length', 'input_sentiment', 'input_sentiment_1_10', 'input_sentiment_1_100', 'cleaned_input_sentiment', 'cleaned_input_sentiment_1_100',
                   'diverse_output_for_model', 'output_length', 'adjusted_output_length',
                   'entity_match_ratio', 'keyword_match_ratio', 'keyword_match_ratio_1_10', 'keyword_match_ratio_1_100',
                   'empathy_count_input', 'politeness_count_input', 'empathy_count_output', 'politeness_count_output',]

In [ ]:
finetuning_data.drop(columns = columns_to_drop, inplace = True)

In [ ]:
# Let's take a look at the trimmed dataset.
finetuning_data.columns

Index(['cleaned_input', 'cleaned_input_sentiment_1_10', 'cleaned_output',
       'issue_complexity', 'customer_sentiment', 'input_text', 'output_text'],
      dtype='object')

In [ ]:
# Turn the data frame into a Hugging Face Dataset & create an 80/20 train-test split.
# Why 80-20? To maintain consistency with the 80/20 split we made for the CSAT score model.
dataset = Dataset.from_pandas(finetuning_data)
dataset = dataset.train_test_split(test_size = 0.2)

train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
'''
In Codebase 3, we consolidated the metadata from the columns below into the input_text and output_text columns, so when we're training the model, we don't need to have these columns.
That streamlines the dataset while maintaining key information, and it makes the model's job easier because now it only has to take in 2 columns instead of 6, and the 2 that it does have to take in have the metadata from the columns below.
'''
columns_to_drop_2 = ['cleaned_input', 'issue_complexity', 'customer_sentiment', 'cleaned_output']

In [ ]:
finetuning_data.drop(columns = columns_to_drop_2, inplace = True)

## Tokenizer Loading & Data Tokenization

In [ ]:
# Load the tokenizer.
tokenizer = T5Tokenizer.from_pretrained('google/flan-t5-large')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
# Tokenize the data.
def tokenize_function(examples):
  model_inputs = tokenizer(
      examples['input_text'], max_length = 512, truncation = True, padding = 'max_length'
  )
  labels = tokenizer(
      examples['output_text'], max_length = 128, truncation = True, padding = 'max_length'
  )

  labels['input_ids'] = [
      [(label if label != tokenizer.pad_token_id else -100) for label in label_ids]
      for label_ids in labels['input_ids']
  ]

  model_inputs['labels'] = labels['input_ids']

  return model_inputs

In [ ]:
# Turn the tokenized data into a Hugging Face Dataset as well.
dataset = Dataset.from_pandas(finetuning_data)
dataset = dataset.train_test_split(test_size = 0.2)

train_dataset = dataset['train']
test_dataset = dataset['test']

In [ ]:
# Apply the tokenization function above to the training & testing subsets we created.
tokenized_train = train_dataset.map(tokenize_function, batched = True)
tokenized_test = test_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/797 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

## Model Loading & Data Collator Setup

In [ ]:
# Load the Flan-T5 model.
model = T5ForConditionalGeneration.from_pretrained('google/flan-t5-large')

In [ ]:
# Add a data collator to account for text length variety by padding the text where necessary to create consistent batch sizes and make the model training process more efficient.
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

## Training Arguments Setup & Model Training

In [ ]:
# Establish the training arguments that we'll train the model on.
training_args = Seq2SeqTrainingArguments(
    output_dir = './results',
    evaluation_strategy = 'steps',
    eval_steps = 100,
    save_strategy = 'steps',
    save_steps = 100,
    learning_rate = 5e-5,
    lr_scheduler_type = 'linear',
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 8,
    weight_decay = 0.05,
    save_total_limit = 3,
    num_train_epochs = 10,
    predict_with_generate = True,
    fp16 = False,
    warmup_steps = 25,
    logging_dir = './logs',
    logging_steps = 100,
    gradient_accumulation_steps = 4,
    load_best_model_at_end = True,
    metric_for_best_model = 'eval_loss'
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Let's do a quick inference test on the untrained model.
input_text = finetuning_data['input_text'][0]
input_ids = tokenizer.encode(input_text, return_tensors = 'pt', max_length = 512, truncation = True)

outputs = model.generate(input_ids, max_length = 128)
predicted_text = tokenizer.decode(outputs[0], skip_special_token = True)

print('Input text: ', input_text)
print('Predicted output: ', predicted_text)

Input text:  Customer Issue:
Hi Tom, I'm trying to log in to my account to purchase an Oven Toaster Grill (OTG), but I'm unable to proceed as it's asking for mobile number or email verification. Can you help me with that? My registered mobile number is +1 123-456-7890. Oh, I'm sorry. I might have registered with a different number. Can you please check with my email address instead? It's johndoe@email.com. Okay, I received the code. What do I do with it? Okay, I entered the code, and it's verified now. Thank you for your help. No, that's all. Thank you.

Agent Response:
Sure, I can assist you with that. May I know your registered mobile number or email address, please? Thank you. Let me check that for you. I'm sorry to inform you that we don't have this number on our records. Can you please confirm if this is the correct number? Sure, let me check that for you. (After a few moments) I see that we have your email address on our records. We'll be sending you a verification code shortly. 

In [ ]:
# Set up the trainer.
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_train,
    eval_dataset = tokenized_test,
    tokenizer = tokenizer,
    data_collator = data_collator,
)

<ipython-input-19-30801c8efa46>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# API Key from Weights & Biases: ****************************************

In [ ]:
# Train the model.
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: architchawla (architchawla-). Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss
100,0.887600,0.169365
200,0.186000,0.152187
300,0.164200,0.149393
400,0.154700,0.144551
500,0.152700,0.144603


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=500, training_loss=0.3090324878692627, metrics={'train_runtime': 2499.6079, 'train_samples_per_second': 3.189, 'train_steps_per_second': 0.2, 'total_flos': 1.836900056825856e+16, 'train_loss': 0.3090324878692627, 'epoch': 10.0})

In [ ]:
# Save the model to a zip file.
model.save_pretrained('./results/Conversation_Evaluation_Flan_T5_Model_1')
tokenizer.save_pretrained('./results/Conversation_Evaluation_Flan_T5_Model_1')
!zip -r Conversation_Evaluation_Flan_T5_Model_1.zip ./results/Conversation_Evaluation_Flan_T5_Model_1

  adding: results/Conversation_Evaluation_Flan_T5_Model_1/ (stored 0%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/config.json (deflated 48%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/model.safetensors (deflated 7%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/added_tokens.json (deflated 83%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/tokenizer_config.json (deflated 94%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/special_tokens_map.json (deflated 85%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/generation_config.json (deflated 29%)
  adding: results/Conversation_Evaluation_Flan_T5_Model_1/spiece.model (deflated 48%)


## Model Reloading & Testing

In [ ]:
# Relaod the model for testing.
model_path = '/content/results/Conversation_Evaluation_Flan_T5_Model_1'
# model_path = '/content/Model'
loaded_model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

model.eval()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [ ]:
# Create some test inputs.

# test_inputs = [
#     "Customer Issue: Hi, I'm having trouble logging into my account. It says my password is incorrect, even though I'm sure it's right.\nAgent Response: Sure, let me check that for you. Could you try resetting the password using the link I provided?\nEvaluate the conversation and summarize the agent's performance:",
#     "Customer Issue: I recently purchased a laptop, but I'm not happy with its performance. Can you help me return it for a refund?\nAgent Response: Sure, I can help with that. Please visit our return center and follow the instructions.\nEvaluate the conversation and summarize the agent's performance:",
#     "Customer Issue: I ordered a kitchen appliance, but it arrived damaged. I tried contacting support before, but didn't get a proper response. This is really frustrating. Can you help me get a replacement or refund?\nAgent Response: I apologize for the inconvenience. I'll initiate a replacement for you right away. You'll receive an email with the details.\nEvaluate the conversation and summarize the agent's performance:",
# ]

test_inputs = [
    "Customer Issue: My internet speed has been incredibly slow for the past few days, and it's getting really frustrating. Can you do something about it?\nAgent Response: I understand your frustration. Let me run a diagnostic test on our end to see if there's an issue with your connection.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I was charged twice for my subscription this month. Can you explain why this happened?\nAgent Response: I apologize for the inconvenience. It seems there was a technical issue on our side. I'll process a refund for the duplicate charge right away.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: The TV I ordered arrived with a cracked screen. This is unacceptable, and I need a replacement or a full refund.\nAgent Response: I sincerely apologize for the inconvenience. I'll initiate the return and replacement process immediately, and you'll receive a confirmation email shortly.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I'm trying to reset my password, but I'm not receiving the password reset email. What should I do?\nAgent Response: I'm sorry about that. Let me verify your email address to make sure it's correct, and I'll resend the password reset link.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I'd like to cancel my account, but I can't find the cancellation option anywhere on your website.\nAgent Response: I can help with that. I will guide you through the steps to cancel your account, or I can cancel it for you if you prefer.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I just realized that I entered the wrong shipping address for my order. Can you correct it?\nAgent Response: Thank you for letting us know. I will update your shipping address now, as long as the order hasn't already shipped. Let me check on that for you.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I tried canceling my subscription last month, but it looks like I'm still being charged. Why is that?\nAgent Response: I apologize for the confusion. Let me look into your account to verify if the cancellation was processed correctly and assist you with a refund if necessary.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: My package was supposed to be delivered three days ago, but I haven't received it yet. Can you find out what's happening?\nAgent Response: I apologize for the delay. Let me contact the courier and check on the status of your package, and I'll provide an update as soon as possible.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: I requested a refund two weeks ago, but I haven't seen it processed yet. When can I expect it?\nAgent Response: I'm really sorry for the delay. Let me check the status of your refund, and I'll provide you with an estimated time for it to be processed.\nEvaluate the conversation and summarize the agent's performance:",

    "Customer Issue: The vacuum cleaner I bought stopped working after just two weeks. This is very disappointing. What can you do about it?\nAgent Response: I'm really sorry about that. We take product quality seriously. I can help you with a replacement or guide you on how to claim the warranty.\nEvaluate the conversation and summarize the agent's performance:",
]

In [ ]:
# Prepare the test inputs above for the model to evaluate them.

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

input_encodings = tokenizer(test_inputs, padding = True, truncation = True, max_length = 512, return_tensors = 'pt')
input_encodings = {key: value.to(device) for key, value in input_encodings.items()}

In [ ]:
# Establish a context manager to disable gradient calculation & tell the model to generate output for the test inputs above.

with torch.no_grad():
  generated_ids = model.generate(
      input_encodings['input_ids'],
      max_length = 128,
      num_beams = 5,
      temperature = 0.7,
      top_p = 0.9,
      repetition_penalty = 1.2,
      do_sample = True
  )

In [ ]:
# Print the model's outputs for the test inputs.
for i, input_text in enumerate(test_inputs):
  output_text = tokenizer.decode(generated_ids[i], skip_special_tokens = True)
  print(f'Input Text:\n{input_text}')
  print(f'\nOutput Text:\n{output_text}\n')

Input Text:
Customer Issue: My internet speed has been incredibly slow for the past few days, and it's getting really frustrating. Can you do something about it?
Agent Response: I understand your frustration. Let me run a diagnostic test on our end to see if there's an issue with your connection.
Evaluate the conversation and summarize the agent's performance:

Output Text:
The customer issue is moderately complex, requiring step-by-step assistance. The agent's response was empathetic and acknowledged the customer's difficulty. The agent’s directions were simple and easy to understand. The agent's tone resonated well with the customer's mood.

Input Text:
Customer Issue: I was charged twice for my subscription this month. Can you explain why this happened?
Agent Response: I apologize for the inconvenience. It seems there was a technical issue on our side. I'll process a refund for the duplicate charge right away.
Evaluate the conversation and summarize the agent's performance:

Output 